In [1]:
import pandas as pd
from datetime import datetime
import csv
import os

class Produccion:
    def __init__(self):
        # Mantiene un registro de todas las órdenes y sus etapas con tiempos
        # Formato: lista de dicts {orden_id, etapa, inicio, fin, duracion}
        self.registros = []
        self.etapas = ["Corte", "Costura", "Acabado"]  # Etapas estándar del proceso

    def registrar_orden(self, orden_id):
        print(f"\nRegistro de tiempos para la orden: {orden_id}")
        for etapa in self.etapas:
            print(f"\n--- Etapa: {etapa} ---")
            inicio = self._pedir_tiempo(f"Ingrese hora de inicio (formato HH:MM, 24h) para etapa '{etapa}': ")
            fin = self._pedir_tiempo(f"Ingrese hora de fin (formato HH:MM, 24h) para etapa '{etapa}': ")

            # Validar que fin sea posterior a inicio
            while fin <= inicio:
                print("Error: La hora de fin debe ser posterior a la de inicio. Intente de nuevo.")
                fin = self._pedir_tiempo(f"Ingrese hora de fin (formato HH:MM, 24h) para etapa '{etapa}': ")

            duracion = (fin - inicio).total_seconds() / 60  # duración en minutos

            registro = {
                "orden_id": orden_id,
                "etapa": etapa,
                "inicio": inicio.strftime("%H:%M"),
                "fin": fin.strftime("%H:%M"),
                "duracion_min": duracion
            }
            self.registros.append(registro)
            print(f"Registrado: {duracion:.2f} minutos en etapa '{etapa}'.")

    def _pedir_tiempo(self, mensaje):
        while True:
            entrada = input(mensaje)
            try:
                hora = datetime.strptime(entrada, "%H:%M")
                # Asegurar que el tiempo sea datetime con fecha arbitraria para comparación
                return hora
            except ValueError:
                print("Formato inválido. Por favor ingrese la hora como HH:MM (ejemplo: 14:30).")

    def mostrar_reporte(self):
        if not self.registros:
            print("\nNo hay registros para mostrar.")
            return

        df = pd.DataFrame(self.registros)
        print("\nReporte general de producción:\n")
        print(df)

        # Estadísticas generales por etapa
        print("\nEstadísticas de duración por etapa (minutos):")
        estadisticas = df.groupby('etapa')['duracion_min'].describe()
        print(estadisticas)

        # Estadísticas por orden (total de minutos por orden)
        print("\nDuración total por orden (minutos):")
        por_orden = df.groupby('orden_id')['duracion_min'].sum()
        print(por_orden)

    def exportar_csv(self, nombre_archivo="reporte_tiempos.csv"):
        if not self.registros:
            print("No hay datos para exportar.")
            return
        df = pd.DataFrame(self.registros)
        df.to_csv(nombre_archivo, index=False)
        print(f"Reporte exportado a {nombre_archivo}")

def menu():
    produccion = Produccion()
    print("=== Sistema de Gestión de Tiempos de Producción - Taller de Confección ===")
    while True:
        print("\nOpciones:")
        print("1. Registrar nueva orden de producción")
        print("2. Mostrar reporte general")
        print("3. Exportar reporte a CSV")
        print("4. Salir")
        opcion = input("Seleccione una opción (1-4): ")

        if opcion == "1":
            orden_id = input("Ingrese el ID o nombre de la orden: ").strip()
            if orden_id == "":
                print("El ID no puede estar vacío.")
                continue
            produccion.registrar_orden(orden_id)
        elif opcion == "2":
            produccion.mostrar_reporte()
        elif opcion == "3":
            nombre = input("Ingrese nombre de archivo para exportar (default: reporte_tiempos.csv): ").strip()
            if nombre == "":
                nombre = "reporte_tiempos.csv"
            produccion.exportar_csv(nombre)
        elif opcion == "4":
            print("Saliendo del sistema. ¡Hasta luego!")
            break
        else:
            print("Opción no válida. Intente de nuevo.")

if __name__ == "__main__":
    menu()



=== Sistema de Gestión de Tiempos de Producción - Taller de Confección ===

Opciones:
1. Registrar nueva orden de producción
2. Mostrar reporte general
3. Exportar reporte a CSV
4. Salir


KeyboardInterrupt: Interrupted by user

In [ ]:
import pandas as pd
from datetime import datetime

class Produccion:
    def __init__(self):
        self.registros = []
        self.etapas = ["Corte", "Costura", "Acabado"]

    def registrar_orden(self, orden_id):
        print(f"\nRegistro de tiempos para la orden: {orden_id}")
        for etapa in self.etapas:
            print(f"\n--- Etapa: {etapa} ---")
            inicio = self._pedir_tiempo(f"Ingrese hora de inicio (formato HH:MM, 24h) para etapa '{etapa}': ")
            fin = self._pedir_tiempo(f"Ingrese hora de fin (formato HH:MM, 24h) para etapa '{etapa}': ")

            while fin <= inicio:
                print("Error: La hora de fin debe ser posterior a la de inicio. Intente de nuevo.")
                fin = self._pedir_tiempo(f"Ingrese hora de fin (formato HH:MM, 24h) para etapa '{etapa}': ")

            duracion = (fin - inicio).total_seconds() / 60  # duración en minutos

            pausa = self._pedir_pausa(f"Ingrese tiempo de pausa (en minutos) para etapa '{etapa}' (0 si no hubo pausa): ")
            duracion_total = duracion - pausa  # Restar pausas de la duración total

            registro = {
                "orden_id": orden_id,
                "etapa": etapa,
                "inicio": inicio.strftime("%H:%M"),
                "fin": fin.strftime("%H:%M"),
                "duracion_min": duracion,
                "pausa_min": pausa,
                "duracion_total_min": duracion_total
            }
            self.registros.append(registro)
            print(f"Registrado: {duracion:.2f} minutos en etapa '{etapa}' con {pausa} minutos de pausa. Duración total: {duracion_total:.2f} minutos.")

    def _pedir_tiempo(self, mensaje):
        while True:
            entrada = input(mensaje)
            try:
                hora = datetime.strptime(entrada, "%H:%M")
                return hora
            except ValueError:
                print("Formato inválido. Por favor ingrese la hora como HH:MM (ejemplo: 14:30).")

    def _pedir_pausa(self, mensaje):
        while True:
            entrada = input(mensaje)
            try:
                pausa = float(entrada)
                if pausa < 0:
                    raise ValueError
                return pausa
            except ValueError:
                print("Por favor ingrese un número válido para la pausa (0 o más).")

    def mostrar_reporte(self):
        if not self.registros:
            print("\nNo hay registros para mostrar.")
            return

        df = pd.DataFrame(self.registros)
        print("\nReporte general de producción:\n")
        print(df)

        print("\nEstadísticas de duración por etapa (minutos):")
        # Cambiar selección para evitar advertencia de pandas
        estadisticas = df.groupby('etapa')[['duracion_min', 'pausa_min', 'duracion_total_min']].describe()
        print(estadisticas)

        print("\nDuración total por orden (minutos):")
        por_orden = df.groupby('orden_id')['duracion_total_min'].sum()
        print(por_orden)

    def exportar_csv(self, nombre_archivo="reporte_tiempos.csv"):
        if not self.registros:
            print("No hay datos para exportar.")
            return
        df = pd.DataFrame(self.registros)
        df.to_csv(nombre_archivo, index=False)
        print(f"Reporte exportado a {nombre_archivo}")

def menu():
    produccion = Produccion()
    print("=== Sistema de Gestión de Tiempos de Producción - Taller de Confección ===")
    while True:
        print("\nOpciones:")
        print("1. Registrar nueva orden de producción")
        print("2. Mostrar reporte general")
        print("3. Exportar reporte a CSV")
        print("4. Salir")
        opcion = input("Seleccione una opción (1-4): ")

        if opcion == "1":
            orden_id = input("Ingrese el ID o nombre de la orden: ").strip()
            if orden_id == "":
                print("El ID no puede estar vacío.")
                continue
            produccion.registrar_orden(orden_id)
        elif opcion == "2":
            produccion.mostrar_reporte()
        elif opcion == "3":
            nombre = input("Ingrese nombre de archivo para exportar (default: reporte_tiempos.csv): ").strip()
            if nombre == "":
                nombre = "reporte_tiempos.csv"
            produccion.exportar_csv(nombre)
        elif opcion == "4":
            print("Saliendo del sistema. ¡Hasta luego!")
            break
        else:
            print("Opción no válida. Intente de nuevo.")

if __name__ == "__main__":
    menu()



=== Sistema de Gestión de Tiempos de Producción - Taller de Confección ===

Opciones:
1. Registrar nueva orden de producción
2. Mostrar reporte general
3. Exportar reporte a CSV
4. Salir

Registro de tiempos para la orden: 1

--- Etapa: Corte ---
Registrado: 45.00 minutos en etapa 'Corte' con 9.0 minutos de pausa. Duración total: 36.00 minutos.

--- Etapa: Costura ---
Registrado: 180.00 minutos en etapa 'Costura' con 15.0 minutos de pausa. Duración total: 165.00 minutos.

--- Etapa: Acabado ---
Registrado: 20.00 minutos en etapa 'Acabado' con 0.0 minutos de pausa. Duración total: 20.00 minutos.

Opciones:
1. Registrar nueva orden de producción
2. Mostrar reporte general
3. Exportar reporte a CSV
4. Salir

Reporte general de producción:

  orden_id    etapa inicio    fin  duracion_min  pausa_min  duracion_total_min
0        1    Corte  09:00  09:45          45.0        9.0                36.0
1        1  Costura  10:00  13:00         180.0       15.0               165.0
2        1  Acab